# 任意のクエリでTF-IDF求める

In [78]:
import copy
import functools as ft
import json
import os
import random as rd

from gensim import corpora
from gensim import models
from operator import itemgetter
from operator import itemgetter

import controll_text as ct

In [86]:
files = os.listdir('mayfes/json_data/1stLayer/')
querys = []
for file in files:
    query,_ = os.path.splitext(file)
    if query.startswith('.'):
        pass
    else:
        querys.append(query)

In [88]:
picked_number = 10
random_querys = rd.sample(querys,picked_number)
for query in random_querys:
    querys.remove(query)
print('今回のクエリ: ' + str(random_querys))
print('残りクエリ数: ' + str(len(querys)))


query_texts = dict()
for query in random_querys:
    row_data = open('mayfes/json_data/1stLayer/{0}.json'.format(query), 'r')
    data = json.load(row_data)
    text_list = list(map(lambda d: ft.reduce(lambda l1, l2: l1+l2, d["texts"]), filter(lambda d: d["texts"] !=[], data)))
    query_texts[query] = text_list
    

noun_conditions = ct.make_noun_conditions(except_nouns=['/', '(', ')', '>', '<', '-', '.','"','_', 'ー','＿','ー',';',':','·','...'])
verb_conditions = ct.make_verb_conditions(except_verbs=[], verb_types = [])
query_words = {}
for k, v in query_texts.items():
    query_words[k] = list(map(ft.partial(ct.extract_words, noun_conditions,verb_conditions), v))
# query_words["サイト"][0:1]



def tfidf(full_texts, target_words_list):
    dictionary = corpora.Dictionary(full_texts)
    n_words = len(dictionary.token2id)
    id2token = {v: k for k, v in dictionary.token2id.items()}
    full_corpus = map(dictionary.doc2bow, full_texts)
    target_corpus = map(dictionary.doc2bow, target_words_list)
    tfidf_model = models.TfidfModel(copy.deepcopy(full_corpus))
    tfidf_vectors_by_tuple = list(tfidf_model[copy.deepcopy(target_corpus)])

    # TF-IDF行列を算出
    vectors = []
    for v_by_tuple in tfidf_vectors_by_tuple:
        vector = [0]*n_words 
        for t in v_by_tuple:
            vector[t[0]] = t[1]
        vectors.append(vector)
    # 行列を転置
    vectors_t = list(zip(*vectors))

    #単語ごとにTF-IDFベクトルの平均値を求める
    vectors_t2 = [list(it) for it in list(zip(*vectors))]
    ave_vec = list(map(lambda l: sum(l)/len(l), vectors_t2))
    tfidf_ave_dict = {i:v for i, v in enumerate(ave_vec)}

    word_dict = dict()
    for k,v in tfidf_ave_dict.items():
        word_dict[id2token[k]] = v

    return sorted(word_dict.items(), key=itemgetter(1), reverse=True)

# 全文
full_texts = ft.reduce(lambda l1, l2:l1+l2, query_words.values())


# 出力
for query in random_querys:
    target_words_list = query_words[query]
    ranking = tfidf(full_texts, target_words_list)
    f = open('./tfidf_text/1stLayer/{0}.txt'.format(query), 'w')
    for x in ranking:
        f.write(str(x) + "\n")
    f.close()

今回のクエリ: ['駒場', '入学', '特性', '稲', '病院', '主体', '協同', '医療', '消費', '入試']
残りクエリ数: 41
YEYE
